In [2]:
from collections import defaultdict
from pathlib import Path
import json
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch

In [3]:
def get_headers(df):
    keys = list(df.keys())[1:]
    headers = list()
    for key in keys:
        key = '_'.join(key.split('_')[:-2]).lower()
        if key not in headers:
            headers.append(key)
        else:
            break
    assert(len(set(headers)) == len(headers))
    return headers

In [12]:
avg_in_KG = 0
avg_out_KG = 0
avg_in_rank = 0
avg_out_rank = 0
avg_in_MC = 0
avg_out_MC = 0
avg_train_acc = 0
avg_test_acc = 0
avg_qc = 0
empty = True
avg_train_loss = 0
avg_test_loss = 0
i = 0

In [14]:
f = '../experiments/results_date2021-05-14-13-41-09_trial0_ResNet34CIFAR_CIFAR10_Adasmomentum0.9_weight_decay0.0005_beta.xlsx'
df = pd.read_excel(f)
headers = get_headers(df)
print(headers)
df = df.T
in_KG = np.asarray(
    df.iloc[headers.index('in_s') + 1::len(headers),
            :])
#assert(len(in_KG[:, -1]) == 250)
out_KG = np.asarray(
    df.iloc[headers.index('out_s') + 1::len(headers),
            :])
in_rank = np.asarray(
    df.iloc[headers.index('in_rank') + 1::len(headers),
            :])
out_rank = np.asarray(
    df.iloc[headers.index('out_rank') + 1::len(headers),
            :])
in_MC = np.asarray(
    df.iloc[headers.index('in_condition') + 1::len(headers),
            :])
out_MC = np.asarray(
    df.iloc[headers.index('out_condition') + 1::len(headers),
            :])
in_MC[in_MC == 0] = 500
out_MC[out_MC == 0] = 500
avg_in_KG += in_KG
avg_out_KG += out_KG
avg_in_rank += in_rank
avg_out_rank += out_rank
avg_in_MC += in_MC
avg_out_MC += out_MC
QC = np.arctan2((in_KG[-1, :] + out_KG[-1, :]), 
    1 - 1 / ((in_MC[-1, :]  + out_MC[-1, :])))
QC = np.linalg.norm(QC) / np.sqrt(in_KG.shape[1])
avg_qc += QC
tag = 'test_acc1'
if 'test_acc1' not in headers:
    tag = 'test_acc'
    if 'test_acc' not in headers:
        tag = 'acc'
test_acc = np.asarray(
    df.iloc[headers.index(tag) + 1::len(headers), :])
tag = 'train_acc1'
avg_test_acc += test_acc
if 'train_acc1' not in headers:
    tag = 'train_acc'
try:
    train_acc = np.asarray(
        df.iloc[headers.index(tag) + 1::len(headers), :])
    avg_train_acc += train_acc
except Exception:
    avg_train_acc += np.zeros_like(avg_train_acc)
train_loss = np.asarray(
            df.iloc[headers.index('train_loss') + 1::len(headers), :])
avg_train_loss += train_loss
try:
    test_loss = np.asarray(
                df.iloc[headers.index('test_loss') + 1::len(headers), :])
    avg_test_loss += test_loss
except Exception:
    avg_test_loss += np.zeros_like(avg_test_loss)

avg_in_KG /= (i + 1)
avg_out_KG /= (i + 1)
avg_in_rank /= (i + 1)
avg_out_rank /= (i + 1)
avg_in_MC /= (i + 1)
avg_out_MC /= (i + 1)
avg_train_acc /= (i + 1)
avg_test_acc /= (i + 1)
avg_qc /= (i + 1)
avg_train_loss /= (i + 1)
avg_test_loss /= (i + 1)
data = {
    "in_KG": avg_in_KG,
    "out_KG": avg_in_KG,
    "in_rank": avg_in_rank,
    "out_rank": avg_in_rank,
    "in_MC": avg_in_MC,
    "out_MC": avg_in_MC,
    "QC": avg_qc,
    "train_loss": avg_train_loss,
    "test_loss": avg_test_loss,
    "train_acc": avg_train_acc,
    "test_acc": avg_test_acc,
}


['train_acc1', 'train_acc5', 'train_loss', 'in_s', 'out_s', 'fc_s', 'in_rank', 'out_rank', 'fc_rank', 'in_condition', 'out_condition', 'rank_velocity', 'learning_rate', 'test_acc1', 'test_acc5', 'test_loss']


In [15]:
print(data)

2, 1.62024002,
        1.62024002, 1.62024002, 1.62024002, 1.62024002, 1.62024002,
        1.62024002],
       [1.62647995, 1.62647995, 1.62647995, 1.62647995, 1.62647995,
        1.62647995, 1.62647995, 1.62647995, 1.62647995, 1.62647995,
        1.62647995, 1.62647995, 1.62647995, 1.62647995, 1.62647995,
        1.62647995, 1.62647995, 1.62647995, 1.62647995, 1.62647995,
        1.62647995, 1.62647995, 1.62647995, 1.62647995, 1.62647995,
        1.62647995, 1.62647995, 1.62647995, 1.62647995, 1.62647995,
        1.62647995, 1.62647995, 1.62647995, 1.62647995, 1.62647995,
        1.62647995],
       [1.62871994, 1.62871994, 1.62871994, 1.62871994, 1.62871994,
        1.62871994, 1.62871994, 1.62871994, 1.62871994, 1.62871994,
        1.62871994, 1.62871994, 1.62871994, 1.62871994, 1.62871994,
        1.62871994, 1.62871994, 1.62871994, 1.62871994, 1.62871994,
        1.62871994, 1.62871994, 1.62871994, 1.62871994, 1.62871994,
        1.62871994, 1.62871994, 1.62871994, 1.62871994, 1.6

In [16]:

qc = list()
       
QC = np.arctan2((metrics['in_KG'][-1, :] + metrics['out_KG'][-1, :]), 
                            1 - ALPHA / ((metrics['in_MC'][-1, :]  + metrics['out_MC'][-1, :])))
            try:
                QC = np.linalg.norm(QC) / np.sqrt(metrics['in_KG'].shape[1])
                if not np.isnan(QC):
                    qc.append(QC)
                    test_acc = np.max(metrics['test_acc'][:, 0])
                    gap = np.max(metrics['train_acc'][:, 0]) - test_acc
                    accs.append(test_acc)
                    gaps.append(gap)
                    kgs.append(metrics['out_KG'][-1, :])
                    mcs.append(metrics['out_MC'][-1, :])
                    if not dataset_level:
                        axs[0].scatter([QC], [gap], label=opt)
                        axs[1].scatter([QC], [test_acc], label=opt)
            except Exception:
                pass
        qc = np.array(qc)
        gaps = np.array(gaps)
        accs = np.array(accs)
        poly_x = np.linspace(np.min(qc), np.max(qc))

        if dataset_level:
            coeff = np.polyfit(qc, gaps, POLYD)
            poly = np.poly1d(coeff)
            fitted = poly(poly_x)
            axs[0+i].plot(poly_x, fitted, 
                        zorder=100)
            if i == 0:
                axs[0+i].scatter(qc, gaps, label=net, alpha=0.3, zorder=1)
            else:
                axs[0+i].scatter(qc, gaps, alpha=0.3, zorder=1)
            coeff = np.polyfit(qc, accs, POLYD)
            poly = np.poly1d(coeff)
            fitted = poly(poly_x)
            axs[1+i].plot(poly_x, fitted, zorder=100)
            axs[1+i].scatter(qc, accs, alpha=0.3, zorder=1)
        print(net)
        print(np.mean(kgs))
        print(np.mean(mcs))
        print('Spearman', stats.spearmanr(qc, -gaps))
        print('Kendal', stats.kendalltau(qc, -gaps))
        if not dataset_level:
            axs[0].set_title(f'{dataset}-{net}')
            axs[1].set_title(f'{dataset}-{net}')
            axs[0].set_xlabel("QC")
            axs[1].set_xlabel("QC")
            axs[0].set_ylabel("Train-Test Gap")
            axs[1].set_ylabel("Test Accuracy")
#             if dataset == 'CIFAR10':
#                 axs[0].set_ylim(-0.1, 0.2)
#                 axs[1].set_ylim(0.9, 0.98)
#             else:
#                 axs[0].set_ylim(0.0, 0.4)
#                 axs[1].set_ylim(0.6, 0.83)
            axs[0].set_aspect(1. / axs[0].get_data_ratio(), adjustable='box')
            axs[1].set_aspect(1. / axs[1].get_data_ratio(), adjustable='box')
            plt.legend(bbox_to_anchor=(1, 1), ncol=3)
            plt.show()
            plt.close()
    if dataset_level:
        axs[0+i].set_title(f'{dataset}', fontsize=FONTSIZE)
        axs[1+i].set_title(f'{dataset}', fontsize=FONTSIZE)
        axs[0+i].set_xlabel("QC", fontsize=FONTSIZE)
        axs[1+i].set_xlabel("QC", fontsize=FONTSIZE)
        axs[0+i].set_ylabel("Generalization Gap", fontsize=FONTSIZE)
        axs[1+i].set_ylabel("Test Accuracy", fontsize=FONTSIZE)
#         axs[0+i].set_xscale('log')
#         axs[1+i].set_xscale('log')
#         if dataset == 'CIFAR10':
#             axs[0].set_ylim(-0.1, 0.2)
#             axs[1].set_ylim(0.9, 0.98)
#         else:
#             axs[0].set_ylim(0.0, 0.4)
#             axs[1].set_ylim(0.6, 0.83)
        axs[0+i].set_aspect(1. / axs[0+i].get_data_ratio(), adjustable='box')
        axs[1+i].set_aspect(1. / axs[1+i].get_data_ratio(), adjustable='box')
        for j in range(2):
            ax = axs[j+i]
            box = ax.get_position()
            box.y0 += 0.01
            box.y1 += 0.01
            ax.set_position(box)
            ax.grid(True, zorder=0)
    
if dataset_level:
    fig.legend(loc='upper center', bbox_to_anchor=(0.5,1.1), ncol=4, prop={'size': 11.5})
    fig.tight_layout()
    fig.savefig(f'figures/resnets_qc_{POLYD}d.png', bbox_inches='tight')
    plt.show()
    plt.close()
        

AttributeError: 'numpy.ndarray' object has no attribute 'items'